# fGSEA benchmark

This jupyter notebook calculates the speed of fGSEA as well as the p-value consistency. We install the newest version of fGSEA from github.

In [1]:
library(devtools)
install_github("ctlab/fgsea")

Loading required package: usethis




viridisLite  (0.4.1  -> 0.4.2 ) [CRAN]
vctrs        (0.6.1  -> 0.6.2 ) [CRAN]
rlang        (1.1.0  -> 1.1.1 ) [CRAN]
cowplot      (NA     -> 1.1.1 ) [CRAN]
BiocParallel (1.30.4 -> 1.32.6) [CRAN]


Installing 5 packages: viridisLite, vctrs, rlang, cowplot, BiocParallel




The downloaded binary packages are in
	/var/folders/r_/yj45vq792mz5r83pxxqhdz640000gn/T//RtmpNkc10y/downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/private/var/folders/r_/yj45vq792mz5r83pxxqhdz640000gn/T/RtmpNkc10y/remotes16fd3036fcd3/ctlab-fgsea-c9d9b4a/DESCRIPTION’ ... OK
* preparing ‘fgsea’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘fgsea_1.27.0.tar.gz’



In [1]:
library(fgsea)
library(data.table)
library(ggplot2)

library(BiocParallel)
register(SerialParam())

Warning message:
“package ‘BiocParallel’ was built under R version 4.2.2”


In [2]:
sig = read.table("ageing_muscle_gtex.tsv", sep=",", header=T)
signature = sig[,2]
names(signature) = sig[,1]

## Initializing gene set library

The gene set library can be downloaded from Enrichr.

In [4]:
ll = readLines("GO_Biological_Process_2021")
genes = list()
gmt = list()
for(l in ll){
    sp = unlist(strsplit(l, "\t"))
    pname = sp[1]
    gmt[[pname]] = sp[3:length(sp)]
}

## Run fGSEA

In [5]:
st = Sys.time()
fgseaRes <- fgsea(pathways = gmt,
                  stats    = signature,
                  eps      = 0.0,
                  minSize  = 5,
                  maxSize  = 4000)

print(Sys.time() - st)

Time difference of 12.90792 secs


In [15]:
fgseaRes[order(fgseaRes[,2]),]

pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
extracellular matrix organization (GO:0030198),2.537829e-24,1.346826e-20,1.2790340,0.5412495,2.784851,237,"CAPN2 , COL19A1 , HTRA1 , LOXL4 , BGN , A2M , NPNT , GSN , PAPLN , COL25A1 , ADAMTS7 , MFAP2 , COL6A1 , CRTAP , GAS6 , LUM , VCAM1 , FBLN5 , COL6A2 , P3H1 , COL18A1 , COL8A1 , SMOC2 , CYP1B1 , COL1A1 , TIMP1 , COL4A3 , ADAM8 , MMP19 , ITGA9 , COL4A6 , FGF2 , TIMP2 , ADAMTSL2, COL7A1 , COL16A1 , FN1 , LAMA5 , ELN , CTSK , PECAM1 , COL15A1 , TGFB1 , JAM3 , MMP2 , LOX , AGRN , ADAM15 , COL1A2 , LOXL2 , LAMA4 , TNC , ITGB3 , DPT , LAMA3 , COL6A3 , NID2 , VWF , TLL1 , LAMC3 , ITGB4 , CAPNS1 , SERPINH1, APP , ADAMTS10, ADAMTS2 , CD44 , ADAMTS9 , SLC39A8 , ITGA10 , SPARC , COL4A4 , ADAMTS12, ITGAL , WASHC1 , COL21A1 , F11R , ITGA3 , VCAN , PPIB , COL4A5 , ADAMTS6 , LOXL3 , COMP , ITGB8 , COL5A1 , CD47 , POSTN , CTSG , ST7 , MMP16 , COL12A1 , ADAMTSL1, ITGA4 , COL3A1 , ADAMTS13, COL24A1 , HSPG2 , FBN1 , SH3PXD2B, FLRT2 , P3H2 , COL9A2 , LOXL1 , PLOD3 , DCN , JAM2 , RECK , ICAM2 , COL28A1 , PXDN , DDR2 , COL27A1 , ECM2 , ADAMTS3 , COL5A2 , COL8A2 , PDGFB"
ubiquitin-dependent protein catabolic process (GO:0006511),1.294720e-19,3.435538e-16,1.1421912,-0.4524496,-2.471797,327,"GID4 , RNF6 , RNF14 , COP1 , AMFR , AREL1 , FBXO45 , RC3H2 , WWP1 , RNF130 , KCTD21 , MARCHF6, PSMD12 , PSME4 , ZRANB1 , UBAP1 , NEDD4 , CUL3 , VPS37A , RAD23B , PLAA , UBQLN1 , KEAP1 , CDC26 , UBE3C , FEM1A , PCNP , CDC23 , UBXN2A , UBE2L3 , COPS3 , RNF8 , OTUD7B , ELOC , ARNTL , GSK3A , UBE2W , CDC16 , VPS37C , ANKIB1 , USP33 , PSMB5 , NPLOC4 , PSMA1 , KBTBD6 , NTAN1 , PSMD14 , PSMD6 , PPP1R11, RNF111 , KLHL15 , UBE2R2 , KCTD5 , USP9X , VPS4B , RNF216 , UBE2A , BIRC2 , PSMA3 , UBR3 , ITCH , ASB2 , UBE2D3 , UBE2K , FBXW11 , RNF19B , PSMA2 , PINK1 , NDFIP2 , PSMD11 , PSMC2 , PSME3 , SIRT1 , RAD23A , FBXL14 , RNF11 , UBE2D4 , TRIM13 , KLHL8 , RNF4 , UBE2S , UBE4A , UBE3B , TRIM3 , CHFR , SMURF2 , KCTD2 , BTRC , RMND5A , UBQLN4 , UBE2H , PSMC6 , CLOCK , CDC27 , PSMA4 , PSMA6 , UBE4B , PSMD7 , PSMD8 , UBXN1 , PSMD2 , UBR4 , RNF114 , PRKN , PSMC1 , RNF5 , ARIH2 , TOPORS , UBE2D2 , RNF40 , VPS37B , PSMB2 , PSMA5 , UBE2D1 , SIRT2 , KLHL25 , UFD1 , CUL4A , CNOT4 , BFAR , PSMB6 , UBQLN2 , PSMC4 , RNF20 , RNF128 , SKP2 , VPS4A , PSMD3 , KBTBD7 , TOLLIP , ZFAND2A, ARIH1 , UBXN2B , ATE1 , SIAH2 , TRIP12 , RNF144A, SPOP , SYVN1 , ZNRF3 , TBL1X , FBXW4 , ARMC8 , AURKA , RFFL , PSMD1 , BAG6 , UBE3D , TNFAIP1, SPSB1 , PSMA7 , FBXL17 , RNF7 , ERCC8 , TSG101 , RNF168 , SPOPL , ADRM1 , MVB12A , RNF166 , HUWE1 , RNF13 , DCAF11 , VCP , RNF167 , AGAP3 , RC3H1 , NFE2L2 , BUB3 , MVB12B , KCTD6 , FBXO21 , ANAPC4"
external encapsulating structure organization (GO:0045229),1.822495e-18,2.592093e-15,1.1053366,0.5452169,2.688988,170,"COL19A1 , BGN , NPNT , PAPLN , ADAMTS7 , MFAP2 , COL6A1 , LUM , VCAM1 , FBLN5 , COL6A2 , COL18A1 , COL8A1 , SMOC2 , COL1A1 , COL4A3 , MMP19 , ITGA9 , COL4A6 , FGF2 , ADAMTSL2, COL7A1 , COL16A1 , FN1 , LAMA5 , ELN , PECAM1 , COL15A1 , JAM3 , MMP2 , LOX , AGRN , ADAM15 , COL1A2 , LAMA4 , TNC , ITGB3 , LAMA3 , COL6A3 , NID2 , VWF , LAMC3 , ITGB4 , APP , ADAMTS10, ADAMTS2 , CD44 , ADAMTS9 , SLC39A8 , ITGA10 , SPARC , COL4A4 , ADAMTS12, ITGAL , F11R , ITGA3 , VCAN , COL4A5 , ADAMTS6 , COMP , ITGB8 , COL5A1 , CD47 , POSTN , ST7 , MMP16 , ADAMTSL1, ITGA4 , COL3A1 , ADAMTS13, COL24A1 , HSPG2 , FBN1 , COL9A2 , LOXL1 , DCN , JAM2 , RECK , ICAM2 , COL28A1 , PXDN , DDR2 , COL27A1 , ECM2 , ADAMTS3 , COL5A2 , COL8A2 , PDGFB"
protein ubiquitination (GO:0016567),1.953717e-18,2.592093e-15,1.1053366,-0.3961828,-2.256523,463,"CTR9 , RNF6 , RNF14 , RNF139 , RNF10 , AMFR , AREL1 , FBXO45 , RC3H2 , WWP1 , H2BC11 , OTULIN , UBE2Q1 , MARCHF6, PSMD12 , PSME4 , TRIM37 , FBXO6 , NEDD4 , FBXL18 , UBE2QL1, VCPIP1 , RNFT1 , CUL3 , TRIM41 , TRIM44 , BRAP , KBTBD8 , KEAP1 , TRIM26 , UBE3C , PCNP , MARCHF5, H2BC5 , CDC23 , UBE2L3 , CUL1 , RNF41 , RNF8 , PEX13 , TRIM24 , TSPAN17, USP7 , MUL1 , E

# Consistency

Compare consistency of log p-values.

In [16]:
ll = readLines("test/KEGG_2019_Mouse.txt")
genes = list()
gmt = list()
for(l in ll){
    sp = unlist(strsplit(l, "\t"))
    pname = sp[1]
    gmt[[pname]] = sp[3:length(sp)]
}

In [ ]:
res = list()
for(i in 1:10){
    fgseaRes <- fgsea(pathways = gmt,
                      stats    = signature,
                      minSize  = 5,
                      maxSize  = 4000
                      )
    res[[length(res)+1]] = log(fgseaRes[,"pval"])
}

## Calulate error

The calculated p-value error will be used in the p-value consistency benchmark notebook to compare GSEApy, blitzGSEA, and fGSEA

In [44]:
pp=do.call(cbind, res)
print(dim(pp))
ee=(1-cor(pp))
diag(ee) = NaN
paste(unlist(rowMeans(ee,  na.rm=T)), collapse=",")


[1] 296  10


[1] "0.00261250354614742,0.00274601095318889,0.00268698398862981,0.00254896856733903,0.0028239254670881,0.00255107983368626,0.00266219682016753,0.00248607221489015,0.00294916957896707,0.00300309821889459"